Lambda School Data Science

*Unit 4, Sprint 1, Module 4*

---

In [1]:
import tensorflow

In [4]:
from tensorflow import keras

In [6]:
from tensorflow.keras.datasets import mnist

# Topic Modeling (Prepare)


# Latent Dirchilet Allocation Models (Learn)
<a id="#p1"></a>

## Overview
LDA is a "generative probabilistic model". 

Let's play with a modoel available [here](https://lettier.com/projects/lda-topic-modeling/)

## Follow Along

## Challenge 

# Estimating LDA Models with Gensim (Learn)
<a id="#p1"></a>

## Overview
### A Litterary Introduction: *Jane Austen V. Charlotte Bronte*
Despite being born nearly forty years apart, modern fans often pit Jane Austen & Charlotte Bronte against one another in a battle for litterary  supremacy. The battle centers around the topics of education for women, courting, and marriage. The authors' similiar backgrounds naturally draw comparisons, but the modern fascination is probably due to novelility of British women publishing novels during the early 19th century. 

Can we help close a litterary battle for supremacy and simply acknowledge that the authors addressed different topics and deserve to be acknowledged as excellent authors each in their own right?

We're going to apply Latent Dirichlet Allocation a machine learning alogrithm for topic modeling to each of the author's novels to compare the distribution of topics in their novels.

In [ ]:
!pip install gensim

In [1]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
import warnings

warnings.filterwarnings( "ignore")

### Novel Data
I grabbed the novel data pre-split into a bunch of smaller chuncks

In [2]:
path = './data/austen-brontë-split'

### Text Preprocessing
**Activity**: update the function `tokenize` with any technique you have learned so far this week. 

In [3]:
# 1) Plain Python - ''.split command
# 2) Spacy - just the lemmas from the document
# 3) Gensim - simple_preprocess

def tokenize(text):
    "Complete this function"
    
    return [token for token in simple_preprocess(text) if token in STOPWORDS]

In [4]:
import os

def gather_data(path_to_data): 
    data = []
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    data.append(tokenize(str(text)))       
    return data

In [5]:
tokens = gather_data(path)

In [8]:
tokens[0][0:10]

['by', 'and', 'with', 'and', 'seemed', 'to', 'some', 'of', 'the', 'of']

In [9]:
"this is a sample string with a \n newline character".replace('\n', '')

'this is a sample string with a  newline character'

In [ ]:
df['tokens'].head()

## Follow Along

In [6]:
titles = [t[:-4] for t in os.listdir(path) if os.path.isdir(t) == False]

In [11]:
len(titles)

813

In [7]:
STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [13]:
tokenize("Hello World! This a test of the tokenization method")

['hello', 'world', 'test', 'tokenization', 'method']

In [14]:
len(tokens)

813

### Author DataFrame


In [8]:
df = pd.DataFrame(index=titles, data={'tokens':tokens})

In [17]:
df.head()

,tokens
Austen_Emma0000,"[by, and, with, and, seemed, to, some, of, the..."
Austen_Emma0001,"[she, were, here, again, what, it, is, that, e..."
Austen_Emma0002,"[all, every, was, every, in, their, not, and, ..."
Austen_Emma0003,"[of, and, of, which, for, the, last, two, or, ..."
Austen_Emma0004,"[some, of, the, had, you, have, of, the, has, ..."


In [9]:
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df.reset_index()['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df.reset_index()['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [10]:
df['author'] = df['author'].map({'Austen':1, 'CBronte':0})

In [18]:
df.author.value_counts()

0    441
1    372
Name: author, dtype: int64

In [21]:
df.head()

,tokens,author,book,section
Austen_Emma0000,"[by, and, with, and, seemed, to, some, of, the...",1,Emma,0
Austen_Emma0001,"[she, were, here, again, what, it, is, that, e...",1,Emma,1
Austen_Emma0002,"[all, every, was, every, in, their, not, and, ...",1,Emma,2
Austen_Emma0003,"[of, and, of, which, for, the, last, two, or, ...",1,Emma,3
Austen_Emma0004,"[some, of, the, had, you, have, of, the, has, ...",1,Emma,4


### Streaming Documents
Here we use a new pythonic thingy: the `yield` statement in our fucntion. This allows us to iterate over a bunch of documents without actually reading them into memory. You can see how we use this fucntion later on. 

In [11]:
def doc_stream(path):
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    tokens = tokenize(str(text))
                yield tokens

In [12]:
streaming_data = doc_stream(path)

In [26]:
type( streaming_data)

generator

In [76]:
# gather_data => returns a list
# doc_stream => returns a generator

In [ ]:
next( streaming_data) # Returns one document at a time from the generator

### Gensim LDA Topic Modeling

In [13]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary( streaming_data)

In [23]:
id2word.token2id['shark']

2592

In [24]:
id2word[869]

'degree'

In [25]:
id2word.doc2bow(tokenize("This is a sample message Darcy England England England"))

[(2754, 1), (3987, 3), (6602, 1), (6819, 1)]

In [26]:
import sys
sys.getsizeof(id2word)

56

In [27]:
len(id2word.keys())

22096

In [14]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=0.95)  # Nothing below 5 document appearances, or above appearing in 95% of docs

In [29]:
len(id2word.keys())

8103

In [15]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
# Although abstracted away - tokenization IS happening in the doc_stream f(x)
corpus = [id2word.doc2bow(text) for text in doc_stream(path)]  # LIST COMPREHENSION

In [91]:
corpus[15][:10]

[(6, 1),
 (8, 1),
 (13, 1),
 (14, 1),
 (23, 1),
 (27, 2),
 (29, 1),
 (34, 2),
 (35, 1),
 (41, 1)]

In [17]:
lda = LdaMulticore(corpus= corpus,
                   id2word= id2word,
                   random_state= 723812,
                   num_topics= 4,
                   passes= 4,
                   workers= 4
                  )

In [18]:
lda.print_topics()

[(0,
  '0.011*"emma" + 0.008*"miss" + 0.007*"think" + 0.006*"harriet" + 0.006*"know" + 0.006*"thing" + 0.006*"good" + 0.006*"weston" + 0.005*"little" + 0.005*"time"'),
 (1,
  '0.005*"little" + 0.005*"good" + 0.005*"like" + 0.004*"thought" + 0.004*"know" + 0.004*"time" + 0.004*"think" + 0.003*"long" + 0.003*"jane" + 0.003*"room"'),
 (2,
  '0.008*"miss" + 0.007*"elizabeth" + 0.007*"elinor" + 0.005*"marianne" + 0.005*"sister" + 0.005*"time" + 0.005*"little" + 0.005*"think" + 0.005*"know" + 0.005*"soon"'),
 (3,
  '0.006*"little" + 0.006*"like" + 0.004*"thought" + 0.004*"know" + 0.003*"good" + 0.003*"day" + 0.003*"time" + 0.003*"long" + 0.003*"hand" + 0.003*"night"')]

In [95]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [96]:
topics = [' '.join(t[0:5]) for t in words]

In [97]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
little night know like time

------ Topic 1 ------
miss emma jane know thing

------ Topic 2 ------
elizabeth darcy bennet jane bingley

------ Topic 3 ------
shall sir jane love like

------ Topic 4 ------
like rochester little long room

------ Topic 5 ------
bessie little miss like think

------ Topic 6 ------
miss temple helen burns day

------ Topic 7 ------
crimsworth hunsden know ll house

------ Topic 8 ------
hunsden day little letter thought

------ Topic 9 ------
sir face rochester candle heard

------ Topic 10 ------
elinor marianne sister edward mother

------ Topic 11 ------
emma harriet elton knightley good

------ Topic 12 ------
lady miss young know middleton

------ Topic 13 ------
little like heart good morning

------ Topic 14 ------
little like madame john dr

------ Topic 15 ------
monsieur little english thought like



## Challenge 

You will apply an LDA model to a customer review dataset to practice the fitting and estimation of LDA. 

# Interpret LDA Results (Learn)
<a id="#p3"></a>

## Overview

## Follow Along

### Topic Distance Visualization

In [19]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

### Overall Model / Documents

In [100]:
lda[corpus[0]]

[(11, 0.9978096)]

In [101]:
distro = [lda[d] for d in corpus]

In [102]:
distro[0]

[(11, 0.9978096)]

In [ ]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [104]:
len(new_distro)

813

In [105]:
df.head()

,tokens,author,book,section
Austen_Emma0000,"[by, and, with, and, seemed, to, some, of, the...",1,Emma,0
Austen_Emma0001,"[she, were, here, again, what, it, is, that, e...",1,Emma,1
Austen_Emma0002,"[all, every, was, every, in, their, not, and, ...",1,Emma,2
Austen_Emma0003,"[of, and, of, which, for, the, last, two, or, ...",1,Emma,3
Austen_Emma0004,"[some, of, the, had, you, have, of, the, has, ...",1,Emma,4


In [ ]:
df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [107]:
df.head()

,little night know like time,miss emma jane know thing,elizabeth darcy bennet jane bingley,shall sir jane love like,like rochester little long room,bessie little miss like think,miss temple helen burns day,crimsworth hunsden know ll house,hunsden day little letter thought,sir face rochester candle heard,elinor marianne sister edward mother,emma harriet elton knightley good,lady miss young know middleton,little like heart good morning,little like madame john dr,monsieur little english thought like,author
Austen_Emma0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.997810,0.0,0.0,0.0,NaN,Austen
Austen_Emma0001,0.0,0.480884,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.516651,0.0,0.0,0.0,NaN,Austen
Austen_Emma0002,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.997416,0.0,0.0,0.0,NaN,Austen
Austen_Emma0003,0.0,0.000000,0.046777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.951074,0.0,0.0,0.0,NaN,Austen
Austen_Emma0004,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.997835,0.0,0.0,0.0,NaN,Austen


In [108]:
df.groupby('author').mean()

,little night know like time,miss emma jane know thing,elizabeth darcy bennet jane bingley,shall sir jane love like,like rochester little long room,bessie little miss like think,miss temple helen burns day,crimsworth hunsden know ll house,hunsden day little letter thought,sir face rochester candle heard,elinor marianne sister edward mother,emma harriet elton knightley good,lady miss young know middleton,little like heart good morning,little like madame john dr,monsieur little english thought like
author,,,,,,,,,,,,,,,,
Austen,0.000000,0.216950,0.262676,0.002249,0.015431,0.000067,0.001914,0.003753,0.002788,0.000000,0.288920,0.196019,0.006189,0.000000,0.000100,0.029602
CBronte,0.016717,0.009676,0.032819,0.005239,0.387265,0.007459,0.018959,0.030336,0.002263,0.009364,0.014162,0.004187,0.003711,0.029774,0.254882,0.346491


## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text prepocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow


# Selecting the Number of Topics (Learn)
<a id="#p4"></a>

## Overview

## Follow Along

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=40, 
                                                        step=6,
                                                        passes=20)

In [ ]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [80]:
topic_coherence.head()

,coherence_score,num_topics,pass
0,-0.603344,2,0
1,-0.859198,8,0
2,-0.821042,14,0
3,-0.710456,20,0
4,-0.843094,26,0


In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
lda[id2word.doc2bow(tokenize("This is a sample document to score with a topic distribution."))]

## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text prepocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [ ]:
test = "Ned asked me a question about England today."

In [ ]:
doc = nlp(test)

for token in doc:
    print(token.text, token.lemma_, token.pos_)

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

In [89]:
def doc_stream(path):
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            with open(os.path.join(path,f)) as t:
                text = t.read().strip('\n')
                yield text

def get_people(docstream):
    
    ppl = []
    
    for d in docstream:
        
        doc = nlp(d)
        
        for ent in doc.ents:
            
            if ent.label_ == "PERSON":
                ppl.append(ent.lemma_)
                
    return set(ppl)

In [ ]:
%pwd

In [90]:
people = get_people(doc_stream(path))

In [ ]:
list(people)[:50]

In [ ]:
gender_lookup = {'Adele':'female_charc', 'William Crimsworth':'male_charc'}

In [ ]:
doc = nlp(next(doc_stream(path)))

In [ ]:
doc.ents[0].lemma_

# Sources

### *References*
* [Andrew Ng et al paper on LDA](https://ai.stanford.edu/~ang/papers/jair03-lda.pdf)
* On [Coherence](https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf)

### *Resources*

* [Gensim](https://radimrehurek.com/gensim/): Python package for topic modeling, nlp, word vectorization, and few other things. Well maintained and well documented.
* [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): A kind of cookbook for LDA with gensim. Excellent overview, but the you need to be aware of missing import statements and assumed prior knowledge.
* [Chinese Restuarant Process](https://en.wikipedia.org/wiki/Chinese_restaurant_process): That really obscure stats thing I mentioned... 
* [PyLDAvis](https://github.com/bmabey/pyLDAvis): Library for visualizing the topic model and performing some exploratory work. Works well. Has a direct parrell implementation in R as well. 
* [Rare Technologies](https://rare-technologies.com/): The people that made & maintain gensim and a few other libraries.
* [Jane Austen v. Charlotte Bronte](https://www.literaryladiesguide.com/literary-musings/jane-austen-charlotte-bronte-different-alike/)